# Figure S3: Glacier projections for the main basins

In [1]:
import pandas as pd
import xarray as xr 

import os
from tqdm.notebook import tqdm

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

import warnings
warnings.simplefilter("ignore")

os.chdir('/home/rooda/OneDrive/Projects/DeepHydro/')

## Data

In [2]:
dict_basins = {"Y00004842":'Puelo', 
              "Y00004886":'Yelcho', 
              "Y00004938":'Palena', 
              "Y00004983":'Cisnes', 
              "Y00004956":'Aysen', 
              "Y00004957":'Baker', 
              "Y00004946":'Pascua', 
              "Y00004814":'Santa Cruz', 
              "Y00004894":'Grey'}

In [3]:
# historical
runoff     = pd.read_csv("results/runoff/glacier_runoff_historical_all.csv", parse_dates = ["time"], index_col = 0)
runoff     = runoff[runoff.index.year >= 2000]
runoff     = runoff.resample("YS").mean()

melt       = pd.read_csv("results/runoff/glacier_melt_historical_all.csv", parse_dates = ["time"], index_col = 0)
melt       = melt[melt.index.year >= 2000]
melt       = melt.resample("YS").mean()

volume     = pd.read_csv("results/runoff/glacier_volume_historical_all.csv", parse_dates = ["time"], index_col = 0)
volume     = volume[volume.index >= 2000]
volume_ref = volume.loc[2020].rename("volume")
volume     = volume / volume_ref

In [4]:
# future
runoff_future = xr.open_dataset("results/runoff/glacier_runoff_future_all.nc", chunks = "auto").glacier_runoff
runoff_future = runoff_future.resample(time = "YS").mean()
runoff_future_mean = runoff_future.mean(dim = "gcm").load()
runoff_future_std  = runoff_future.std(dim = "gcm").load()

melt_future = xr.open_dataset("results/runoff/glacier_melt_future_all.nc", chunks="auto").melt_on_glacier_daily
melt_future = melt_future.resample(time = "YS").mean()
melt_future_mean = melt_future.mean(dim = "gcm").load()
melt_future_std  = melt_future.std(dim = "gcm").load()

volume_ref    = volume_ref.to_xarray().rename({"index": "rgi_id"})
volume_future = xr.open_dataset("results/runoff/glacier_volume_future_all.nc").volume
volume_future = volume_future / volume_ref
volume_future_mean = volume_future.mean(dim = "gcm")
volume_future_std  = volume_future.std(dim = "gcm")

## Plot

In [5]:
fig    = make_subplots(rows=9, cols=3, horizontal_spacing = 0.05, vertical_spacing = 0.01, 
                       shared_xaxes= True, shared_yaxes= False, row_titles = list(dict_basins.values()),
                       subplot_titles= ["Volume relative to 2020 (%)", 
                                        "Glacier runoff (Q<sub>g</sub><sup>runoff</sup>; m<sup>3</sup> s<sup>-1</sup>)", 
                                        "Glacier melt (Q<sub>g</sub><sup>melt</sup>; m<sup>3</sup> s<sup>-1</sup>)"])

cl = px.colors.qualitative.D3
scen_colors = {"Historical":"rgba(0, 0, 0, 0.8)", "ssp126":cl[0], "ssp585":cl[3]}
shaded_colors = {"Historical":"rgba(0, 0, 0, 0.15)", "ssp126": "rgba(31, 119, 180,0.15)", "ssp585":"rgba(214, 39, 40, 0.15)"}

for x in range(0,9):
    
    # historical
    fig.add_trace(go.Scatter(x=volume.index, y=volume[list(dict_basins)[x]] , mode='lines', name= "Historical", 
                                     line=dict(color=scen_colors["Historical"], width = 1), showlegend=False), row=x+1, col=1)

    fig.add_trace(go.Scatter(x=volume.index, y=runoff[list(dict_basins)[x]] , mode='lines', name= "Historical", 
                                     line=dict(color=scen_colors["Historical"], width = 1), showlegend=False), row=x+1, col=2)
    
    fig.add_trace(go.Scatter(x=volume.index, y=melt[list(dict_basins)[x]] , mode='lines', name= "Historical", 
                                     line=dict(color=scen_colors["Historical"], width = 1), showlegend=False), row=x+1, col=3)

    # future
    for ssp in ["ssp126", "ssp585"]:

        ## volume
        fig.add_trace(go.Scatter(x=volume_future.time.to_series(), y= (volume_future_mean + volume_future_std).sel(ssp = ssp, rgi_id = [list(dict_basins)[x]]).to_series(), 
                                 line=dict(width=0), fillcolor=shaded_colors[ssp], showlegend=False), row=x+1, col=1)
        fig.add_trace(go.Scatter(x=volume_future.time.to_series(), y= (volume_future_mean - volume_future_std).sel(ssp = ssp, rgi_id = [list(dict_basins)[x]]).to_series(), 
                                 line=dict(width=0), fillcolor=shaded_colors[ssp], fill='tonexty', showlegend=False), row=x+1, col=1)
        fig.add_trace(go.Scatter(x=volume_future.time.to_series(), y=volume_future_mean.sel(ssp = ssp, rgi_id = [list(dict_basins)[x]]).to_series(), mode='lines', name= "Historical", 
                                         line=dict(color=scen_colors[ssp], width = 1), showlegend=False), row=x+1, col=1)
    
        ## runoff
        fig.add_trace(go.Scatter(x=pd.DatetimeIndex(runoff_future.time).year, y= (runoff_future_mean + runoff_future_std).sel(ssp = ssp, rgi_id = [list(dict_basins)[x]]).to_series(), 
                                 line=dict(width=0), fillcolor=shaded_colors[ssp], showlegend=False), row=x+1, col=2)
        fig.add_trace(go.Scatter(x=pd.DatetimeIndex(runoff_future.time).year, y= (runoff_future_mean - runoff_future_std).sel(ssp = ssp, rgi_id = [list(dict_basins)[x]]).to_series(), 
                                 line=dict(width=0), fillcolor=shaded_colors[ssp], fill='tonexty', showlegend=False), row=x+1, col=2)
        fig.add_trace(go.Scatter(x=pd.DatetimeIndex(runoff_future.time).year, y=runoff_future_mean.sel(ssp = ssp, rgi_id = [list(dict_basins)[x]]).to_series(), mode='lines', name= "Historical", 
                                         line=dict(color=scen_colors[ssp], width = 1), showlegend=False), row=x+1, col=2)

        ## melt
        fig.add_trace(go.Scatter(x=pd.DatetimeIndex(melt_future.time).year, y= (melt_future_mean + melt_future_std).sel(ssp = ssp, rgi_id = [list(dict_basins)[x]]).to_series(), 
                                 line=dict(width=0), fillcolor=shaded_colors[ssp], showlegend=False), row=x+1, col=3)
        fig.add_trace(go.Scatter(x=pd.DatetimeIndex(melt_future.time).year, y= (melt_future_mean - melt_future_std).sel(ssp = ssp, rgi_id = [list(dict_basins)[x]]).to_series(), 
                                 line=dict(width=0), fillcolor=shaded_colors[ssp], fill='tonexty', showlegend=False), row=x+1, col=3)
        fig.add_trace(go.Scatter(x=pd.DatetimeIndex(melt_future.time).year, y=melt_future_mean.sel(ssp = ssp, rgi_id = [list(dict_basins)[x]]).to_series(), mode='lines', name= "Historical", 
                                         line=dict(color=scen_colors[ssp], width = 1), showlegend=False), row=x+1, col=3)

# legend
fig.add_trace(go.Scatter(x=[0], y= [0], mode='lines', name= "Historical", line=dict(color=scen_colors["Historical"], width = 1), showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=[0], y= [0], mode='lines', name= "SSP 1-2.6", line=dict(color=scen_colors["ssp126"], width = 1), showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=[0], y= [0], mode='lines', name= "SSP 5-8.5", line=dict(color=scen_colors["ssp585"], width = 1), showlegend=True), row=1, col=1)

#  some tweaks
for x in range(0,9):
    fig.update_yaxes(tickformat=".0%", dtick = 0.4, range = [0, 1.29], row=x+1, col=1)

fig.update_yaxes(ticks="outside", griddash = "dot",  zeroline = False)
fig.update_xaxes(range = (2000,2099), ticks="outside", griddash = "dot", dtick = 20)
fig.update_layout(legend=dict(yanchor="top", y=1, xanchor="left", x=0.185, orientation="v", bgcolor = 'rgba(0,0,0,0.0)'))
fig.update_layout(height=1200, width=1050, template = "seaborn", margin = dict(l=10, r=10, b=10, t=30), hovermode = False)

# save figure 
fig.write_image("reports/figures/FigureS3_hydro_projections.png", scale=4)
#fig.show()

## Text

In [ ]:
volume_future = xr.open_dataset("results/runoff/glacier_volume_future_all.nc").volume
volume_future = volume_future.sum("rgi_id") 
volume_future = (1-(volume_future / volume_ref.sum("rgi_id")))*100

In [ ]:
"The volume loss varies from {:.0f} ± {:.0f}% in SSP 1-2.6 to {:.0f} ± {:.0f}% in SSP 5-8.5".format(
    volume_future.mean("gcm").sel(time = 2099).sel(ssp = "ssp126").values,
    volume_future.std("gcm").sel(time = 2099).sel(ssp = "ssp126").values,
    volume_future.mean("gcm").sel(time = 2099).sel(ssp = "ssp585").values,
    volume_future.std("gcm").sel(time = 2099).sel(ssp = "ssp585").values)

In [ ]:
runoff_future_2050 = runoff_future.sum("rgi_id").sel(time = slice("2040-01-01","2059-01-01")).mean("time").load()
runoff_future_2090 = runoff_future.sum("rgi_id").sel(time = slice("2080-01-01","2099-01-01")).mean("time").load()
melt_future_2050   = melt_future.sum("rgi_id").sel(time = slice("2040-01-01","2059-01-01")).mean("time").load()
melt_future_2090   = melt_future.sum("rgi_id").sel(time = slice("2080-01-01","2099-01-01")).mean("time").load()

In [ ]:
"Historical melt mantains until mid century {:.1f} ± {:.1f}% and decreases to {:.1f} ± {:.1f}%  ".format(
    melt_future_2050.mean("gcm").sel(ssp = "ssp585").values * 100 / melt.sum(axis = 1).mean(),
    melt_future_2050.std("gcm").sel(ssp = "ssp585").values * 100 /  melt.sum(axis = 1).mean(), 
    melt_future_2090.mean("gcm").sel(ssp = "ssp585").values * 100 / melt.sum(axis = 1).mean(),
    melt_future_2090.std("gcm").sel(ssp = "ssp585").values * 100 /  melt.sum(axis = 1).mean())

In [ ]:
"Historical runoff mantains until mid century {} and decreases to {} ".format(
    runoff_future_2050.mean("gcm").sel(ssp = "ssp585").values /    runoff.sum(axis = 1).mean(),    
    runoff_future_2090.mean("gcm").sel(ssp = "ssp585").values /    runoff.sum(axis = 1).mean())